In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import os


In [2]:
# SET UP LOGGER
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("stock_data.log"),
        logging.StreamHandler()
    ]       
)
logger = logging.getLogger()

In [3]:
# GET USER TICKERS
def get_user_tickers(n=5):
    tickers = []
    for i in range(n):
        ticker = input(f"Enter in your ticker {i + 1}: ").upper()
        tickers.append(ticker)
    return tickers


In [4]:
# FETCH HISTORICAL DATA FOR EACH TICKER
def fetch_data(tickers, start="2020-01-01", end="2024-12-31"):
    try:
        logger.info(f"Fetching data for tickers: {', '.join(tickers)}")
        raw_data = yf.download(tickers, start=start, end=end)

        if raw_data is None or raw_data.empty:
            raise ValueError("No data fetched. Please check the ticker symbols.")

        logger.debug(f"Raw data columns: {raw_data.columns}")

        # Handle multiple tickers
        if isinstance(raw_data.columns, pd.MultiIndex):
            try:
                df = raw_data.xs('Adj Close', level=0, axis=1)
                logger.info("Using 'Adj Close' for all tickers.")
            except KeyError:
                logger.warning("'Adj Close' not found — falling back to 'Close'")
                df = raw_data.xs('Close', level=0, axis=1)
        else:
            # Single ticker fallback
            if 'Adj Close' in raw_data.columns:
                df = raw_data[['Adj Close']].copy()
                df.columns = [tickers[0]]
            elif 'Close' in raw_data.columns:
                logger.warning("'Adj Close' not found — using 'Close' instead.")
                df = raw_data[['Close']].copy()
                df.columns = [tickers[0]]
            else:
                raise KeyError("Neither 'Adj Close' nor 'Close' found in single-level data.")

        df.dropna(inplace=True)
        logger.info(f"Successfully fetched price data with shape {df.shape}")
        return df

    except Exception as e:
        logger.error(f"Error fetching data: {e}")
        return pd.DataFrame()

In [5]:
# SAVE DATA TO CSV
def save_data(df, path="data/raw_prices.csv"):
    os.makedirs("data", exist_ok=True)
    try:
        df.to_csv(path)
        logger.info(f"Data saved to {path}")
    except Exception as e:
        logger.error(f"Error saving data: {e}")

In [6]:
# MAIN EXECUTION
def main():
    # Get user tickers
    tickers = get_user_tickers()
    df = fetch_data(tickers)

    if df.empty:
        logger.error("No data fetched. Exiting.")
    else:
        print("\nSample of data downloaded:")
        print(df.tail())
        save_data(df)

if __name__ == "__main__":
    main()
    logger.info("Script execution completed.")


2025-04-16 09:16:49,824 - INFO - Fetching data for tickers: AAPL, NVDA, MSFT, JNJ, TSLA


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed
2025-04-16 09:16:51,654 - WARNING - 'Adj Close' not found — falling back to 'Close'
2025-04-16 09:16:51,656 - INFO - Successfully fetched price data with shape (1257, 5)
2025-04-16 09:16:51,683 - INFO - Data saved to data/raw_prices.csv
2025-04-16 09:16:51,684 - INFO - Script execution completed.



Sample of data downloaded:
Ticker            AAPL         JNJ        MSFT        NVDA        TSLA
Date                                                                  
2024-12-23  254.989655  144.116409  434.379028  139.657150  430.600006
2024-12-24  257.916443  144.691803  438.450836  140.207108  462.279999
2024-12-26  258.735504  144.423935  437.233276  139.917130  454.130005
2024-12-27  255.309296  143.898148  429.668457  136.997391  431.660004
2024-12-30  251.923019  142.201721  423.979858  137.477356  417.410004


In [7]:
# CALCULATE DAILY RETURNS FOR EACH ASSET IN PORTFOLIO
def calculate_daily_returns(prices_df):
    """
    This section calculates the daily percentage returns from a DataFrame of prices.

    Parameters:
        prices_df (pd.DataFrame): DataFrame containing price data with dates as index and tickers as columns.

    Returns:
        pd.DataFrame: DataFrame of daily percentage returns with the same index as prices_df.
    """

    returns = prices_df.pct_change().dropna()
    return returns


In [8]:
# TESTING THE FUNCTION
df_prices = pd.read_csv("data/raw_prices.csv", index_col="Date", parse_dates=True)
# print(df_prices.head())

df_returns = calculate_daily_returns(df_prices)
print(df_returns.tail())    

                AAPL       JNJ      MSFT      NVDA      TSLA
Date                                                        
2024-12-23  0.003065  0.005538 -0.003092  0.036897  0.022657
2024-12-24  0.011478  0.003993  0.009374  0.003938  0.073572
2024-12-26  0.003176 -0.001851 -0.002777 -0.002068 -0.017630
2024-12-27 -0.013242 -0.003641 -0.017302 -0.020868 -0.049479
2024-12-30 -0.013263 -0.011789 -0.013240  0.003503 -0.033012


In [9]:
# CALCULATE THE HISTORICAL VALUE AT RISK (VaR) FOR EACH ASSET IN PORTFOLIO
def historical_var(df_returns, alpha=0.05):
    """
    This function calculates the historical Value at Risk (VaR) for a given DataFrame of returns.

    Parameters:
        returns_df (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.
        alpha (float): Significance level for VaR calculation (default is 0.05 for 95% confidence level).

    Returns:
        pd.Series: Series containing the VaR for each asset at the specified confidence level.
    """

    var = df_returns.quantile(1 - alpha)
    return -var # Return as positive value for VaR (losses are negative)

# TESTING THE FUNCTION
var_95 = historical_var(df_returns, alpha=0.05)
print("1 day VaR at 95% confidence level:")
print(var_95)

1 day VaR at 95% confidence level:
AAPL   -0.030369
JNJ    -0.016257
MSFT   -0.027952
NVDA   -0.055386
TSLA   -0.073289
Name: 0.95, dtype: float64


In [10]:
# CALCULATE PARAMETRIC VALUE AT RISK (VaR) FOR EACH ASSET IN PORTFOLIO
from scipy.stats import norm
def parametric_var(df_returns, alpha=0.05):
    """
    This function calculates the parametric Value at Risk (VaR) for a given DataFrame of returns.

    Parameters:
        returns_df (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.
        alpha (float): Significance level for VaR calculation (default is 0.05 for 95% confidence level).

    Returns:
        pd.Series: Series containing the VaR for each asset at the specified confidence level.
    """

    z_score = norm.ppf(1 - alpha)  # Z-score for the given confidence level
    mu = df_returns.mean() # Mean of returns
    sigma = df_returns.std() # Standard deviation of returns

    var_param = -(mu + z_score * sigma)  # VaR formula
    return var_param

# TESTING THE FUNCTION
alpha = 0.05
var_param_95 = parametric_var(df_returns, alpha=0.05)
print ("Parametric VaR at 95% confidence level:")
print(var_param_95)

print("\nInterpreting the results:")
for i in range(len(var_param_95)):
    print(f"There is a {alpha*100}% chance that the price of {df_returns.columns[i]} will drop by more than {var_param_95[i]:.2%} in one day.")

Parametric VaR at 95% confidence level:
AAPL   -0.034024
JNJ    -0.020544
MSFT   -0.032605
NVDA   -0.058907
TSLA   -0.072645
dtype: float64

Interpreting the results:
There is a 5.0% chance that the price of AAPL will drop by more than -3.40% in one day.
There is a 5.0% chance that the price of JNJ will drop by more than -2.05% in one day.
There is a 5.0% chance that the price of MSFT will drop by more than -3.26% in one day.
There is a 5.0% chance that the price of NVDA will drop by more than -5.89% in one day.
There is a 5.0% chance that the price of TSLA will drop by more than -7.26% in one day.


C:\Users\roman\AppData\Local\Temp\ipykernel_53624\405750789.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"There is a {alpha*100}% chance that the price of {df_returns.columns[i]} will drop by more than {var_param_95[i]:.2%} in one day.")


In [11]:
# CALCULATE SHARPE RATIO FOR EACH ASSET IN PORTFOLIO
def calculate_sharpe_ratio(df_returns, risk_free_rate=0.01, annualize=True):
    """
    This function calculates the Sharpe Ratio for a given DataFrame of returns.

    Parameters:
        df_returns (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.
        risk_free_rate (float): Risk-free rate (default is 0.01 for 1%).

    Returns:
        pd.Series: Series containing the Sharpe Ratio for each asset.
    """

    # Convert annual risk-free rate to daily
    rf_daily = (1 + risk_free_rate) ** (1/252) - 1

    excess_returns = df_returns.mean() - rf_daily
    volatility = df_returns.std()

    sharpe_daily = excess_returns / volatility

    if annualize:
        sharpe_annual = sharpe_daily * np.sqrt(252)  # Annualize the Sharpe Ratio
        return sharpe_annual
    else:
        return sharpe_daily
    
# TESTING THE FUNCTION
sharpe_ratios = calculate_sharpe_ratio(df_returns, risk_free_rate=0.01, annualize=True)
print("Annualized Sharpe Ratios:")
print(sharpe_ratios)

# INTERPRET RESULTS
print("\nInterpreting the results:")
for i in range(len(sharpe_ratios)):
    print(f"{df_returns.columns[i]} earns {sharpe_ratios[i]:.2f} units of return per unit of risk annually.")


# FURTHER INTERPRETATION OF SHARPE RATIO
print("\nFurther interpreting the Sharpe Ratio:")
for i in range(len(sharpe_ratios)):
    if sharpe_ratios.iloc[i] < 0:
        print(f"{df_returns.columns[i]} has a negative Sharpe Ratio, indicating it underperforms the risk-free rate.")
    elif sharpe_ratios.iloc[i] < 1:
        print(f"{df_returns.columns[i]} has a Sharpe Ratio less than 1, indicating it earns less return per unit of risk.")
    elif sharpe_ratios.iloc[i] < 2:
        print(f"{df_returns.columns[i]} has a Sharpe Ratio between 1 and 2, indicating it earns a reasonable return per unit of risk.")
    else:
        print(f"{df_returns.columns[i]} has a Sharpe Ratio greater than 2, indicating it earns a high return per unit of risk.")

Annualized Sharpe Ratios:
AAPL    0.913862
JNJ     0.170327
MSFT    0.789172
NVDA    1.417582
TSLA    1.119999
dtype: float64

Interpreting the results:
AAPL earns 0.91 units of return per unit of risk annually.
JNJ earns 0.17 units of return per unit of risk annually.
MSFT earns 0.79 units of return per unit of risk annually.
NVDA earns 1.42 units of return per unit of risk annually.
TSLA earns 1.12 units of return per unit of risk annually.

Further interpreting the Sharpe Ratio:
AAPL has a Sharpe Ratio less than 1, indicating it earns less return per unit of risk.
JNJ has a Sharpe Ratio less than 1, indicating it earns less return per unit of risk.
MSFT has a Sharpe Ratio less than 1, indicating it earns less return per unit of risk.
NVDA has a Sharpe Ratio between 1 and 2, indicating it earns a reasonable return per unit of risk.
TSLA has a Sharpe Ratio between 1 and 2, indicating it earns a reasonable return per unit of risk.


C:\Users\roman\AppData\Local\Temp\ipykernel_53624\3901875079.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{df_returns.columns[i]} earns {sharpe_ratios[i]:.2f} units of return per unit of risk annually.")


In [12]:
# CALCULATE MAXIMUM DRAWDOWN FOR EACH ASSET IN PORTFOLIO
def calculate_max_drawdown(df_prices):
    """
    This function calculates the maximum drawdown for a given DataFrame of prices.

    Parameters:
        df_prices (pd.DataFrame): DataFrame containing price data with dates as index and tickers as columns.

    Returns:
        pd.Series: Series containing the maximum drawdown for each asset.
    """

    drawdowns = {}

    for ticker in df_prices.columns:
        prices = df_prices[ticker]
        rolling_max = prices.cummax()
        drawdown = (prices / rolling_max) - 1  # Calculate drawdown
        max_drawdown = drawdown.min()
        drawdowns[ticker] = abs(max_drawdown)  # Store as positive value
    
    return pd.Series(drawdowns)

# TESTING THE FUNCTION
max_dd = calculate_max_drawdown(df_prices)
print("Maximum Drawdown:")
print(max_dd)

# INTERPRET RESULTS
print("\nInterpreting the results:")
for i in range(len(max_dd)):
    print(f"{df_prices.columns[i]} experienced a maximum drawdown of {max_dd[i]:.2%} from its peak price to trough.")

# FURTHER INTERPRETATION OF MAXIMUM DRAWDOWN
print("\nFurther interpreting the Maximum Drawdown:")
for i in range(len(max_dd)):
    if max_dd[i] < 0.1:
        print(f"{df_prices.columns[i]} had a maximum drawdown of less than 10%, indicating relatively stable performance.")
    elif max_dd[i] < 0.2:
        print(f"{df_prices.columns[i]} had a maximum drawdown between 10% and 20%, indicating moderate volatility.")
    elif max_dd[i] < 0.3:
        print(f"{df_prices.columns[i]} had a maximum drawdown between 20% and 30%, indicating significant volatility.")
    else:
        print(f"{df_prices.columns[i]} had a maximum drawdown greater than 30%, indicating high volatility and risk.")

 


Maximum Drawdown:
AAPL    0.314273
JNJ     0.273663
MSFT    0.371485
NVDA    0.663351
TSLA    0.736322
dtype: float64

Interpreting the results:
AAPL experienced a maximum drawdown of 31.43% from its peak price to trough.
JNJ experienced a maximum drawdown of 27.37% from its peak price to trough.
MSFT experienced a maximum drawdown of 37.15% from its peak price to trough.
NVDA experienced a maximum drawdown of 66.34% from its peak price to trough.
TSLA experienced a maximum drawdown of 73.63% from its peak price to trough.

Further interpreting the Maximum Drawdown:
AAPL had a maximum drawdown greater than 30%, indicating high volatility and risk.
JNJ had a maximum drawdown between 20% and 30%, indicating significant volatility.
MSFT had a maximum drawdown greater than 30%, indicating high volatility and risk.
NVDA had a maximum drawdown greater than 30%, indicating high volatility and risk.
TSLA had a maximum drawdown greater than 30%, indicating high volatility and risk.


C:\Users\roman\AppData\Local\Temp\ipykernel_53624\3520932235.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{df_prices.columns[i]} experienced a maximum drawdown of {max_dd[i]:.2%} from its peak price to trough.")
C:\Users\roman\AppData\Local\Temp\ipykernel_53624\3520932235.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if max_dd[i] < 0.1:
C:\Users\roman\AppData\Local\Temp\ipykernel_53624\3520932235.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a v

In [13]:
# CALCULATE ROLLING VOLATILITY FOR EACH ASSET IN PORTFOLIO
def rolling_volatility(df_returns, window=20):
    """
    This function calculates the rolling volatility for a given DataFrame of returns.

    Parameters:
        df_returns (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.
        window (int): Rolling window size (default is 20 days).

    Returns:
        pd.DataFrame: DataFrame containing the rolling volatility for each asset.
    """

    rolling_vol = df_returns.rolling(window).std() * np.sqrt(252)  # Annualize the volatility
    return rolling_vol

# TESTING THE FUNCTION
rolling_vol = rolling_volatility(df_returns, window=20)
print("Rolling Volatility (20-day window):")
print(rolling_vol.tail())

# INTERPRET RESULTS
print("\nInterpreting the results:")
for i in range(len(rolling_vol.columns)):
    print(f"The rolling volatility for {rolling_vol.columns[i]} is {rolling_vol.iloc[-1, i]:.2%} over the last 20 days.")
    

Rolling Volatility (20-day window):
                AAPL       JNJ      MSFT      NVDA      TSLA
Date                                                        
2024-12-23  0.141174  0.145646  0.200234  0.367499  0.595966
2024-12-24  0.140134  0.145684  0.201738  0.333764  0.604523
2024-12-26  0.139406  0.145385  0.188727  0.333502  0.613655
2024-12-27  0.152639  0.141766  0.194785  0.339849  0.647214
2024-12-30  0.162911  0.144786  0.201190  0.331227  0.661656

Interpreting the results:
The rolling volatility for AAPL is 16.29% over the last 20 days.
The rolling volatility for JNJ is 14.48% over the last 20 days.
The rolling volatility for MSFT is 20.12% over the last 20 days.
The rolling volatility for NVDA is 33.12% over the last 20 days.
The rolling volatility for TSLA is 66.17% over the last 20 days.


In [14]:
# MEASURE PORTFOLIO PERFORMANCE
def portfolio_performance(df_returns, weights):
    """
    This function calculates the expected return and volatility of a portfolio.

    Parameters:
        df_returns (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.
        weights (list): List of weights for each asset in the portfolio.

    Returns:
        tuple: Expected return and volatility of the portfolio.
    """

    # Ensure weights sum to 1
    if not np.isclose(np.sum(weights), 1):
        raise ValueError("Weights must sum to 1.")

    # Calculate expected return and volatility
    expected_return = np.dot(weights, df_returns.mean()) * 252  # Annualize the return
    expected_volatility = np.sqrt(np.dot(weights, np.dot(df_returns.cov() * 252, weights)))  # Annualize the volatility

    return expected_return, expected_volatility




In [15]:
# DEFINE PORTFOLIO WEIGHTS USING MINIMUM VARIANCE OPTIMIZATION
from scipy.optimize import minimize

def min_variance_portfolio(df_returns):
    """
    This function calculates the minimum variance portfolio weights using historical return covariance.

    Parameters:
        df_returns (pd.DataFrame): DataFrame containing daily returns with dates as index and tickers as columns.

    Returns:
        list: List of weights for the minimum variance portfolio.
    """
    n_assets = df_returns.shape[1]
    cov_matrix = df_returns.cov() * 252  # Annualize the covariance matrix

    # Initial guess for weights
    init_guess = np.ones(n_assets) / n_assets

    # Constraints: weights sum to 1
    constraints = ({'type' : 'eq', 'fun': lambda w: np.sum(w) - 1})

    # Bounds for weights: between 0 and 1
    bounds = [(0.0, 1.0)] * n_assets

    # Optimization function to minimize portfolio variance
    def portfolio_volatility(w):
        return np.sqrt(np.dot(w, np.dot(cov_matrix * 252, w)))
    
    result = minimize(portfolio_volatility, init_guess, method='SLSQP', bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError("Optimization failed.", result.message)
    
    return result.x

# TESTING THE FUNCTION
optimal_weights = min_variance_portfolio(df_returns)

for i in range(len(optimal_weights)):
    print(f"Optimal weight for {df_returns.columns[i]}: {optimal_weights[i]:.2%}")

# CALCULATE EXPECTED RETURN AND VOLATILITY OF THE MINIMUM VARIANCE PORTFOLIO
expected_return, expected_volatility = portfolio_performance(df_returns, optimal_weights)
print(f"\nExpected return of the minimum variance portfolio: {expected_return:.2%}")
print(f"Expected volatility of the minimum variance portfolio: {expected_volatility:.2%}")



Optimal weight for AAPL: 5.94%
Optimal weight for JNJ: 80.34%
Optimal weight for MSFT: 11.54%
Optimal weight for NVDA: 0.00%
Optimal weight for TSLA: 2.18%

Expected return of the minimum variance portfolio: 9.82%
Expected volatility of the minimum variance portfolio: 18.79%


In [16]:
# TRANSLATION OF THE RESULTS
def explain_min_var_portfolio(weights, tickers, expected_return, expected_volatility, threshold=0.1):
    """
    This function explains the minimum variance portfolio results.

    Parameters:
        weights (list or np.ndarray): Optimal asset weights
        tickers (list): List of asset names
        expected_return (float): Portfolio return (annualized)
        expected_volatility (float): Portfolio volatility (annualized)
        threshold (float): Weight threshold to determine major vs minor holdings
    """
    explanation = []

    # Identify major and minor holdings
    major_assets = [(tickers[i], weights[i]) for i in range(len(weights)) if weights[i] > threshold]
    minor_assets = [(tickers[i], weights[i]) for i in range(len(weights)) if weights[i] <= threshold]

    explanation.append("Major Holdings:")
    for name, w in major_assets:
        explanation.append(f"• {name} makes up {w:.2%} of the portfolio — likely due to its lower volatility or better diversification.")
    for name, w in minor_assets:
        explanation.append(f"• {name} makes up {w:.2%} of the portfolio — likely due to its higher volatility or lower diversification benefits.")

    explanation.append("\nExpected Portfolio Performance:")
    explanation.append(f"The expected annual return is {expected_return:.2%}, with an annualized volatility of {expected_volatility:.2%}.")

    if expected_volatility < 0.2:
        explanation.append("This indicates a relatively low-risk portfolio, applicable for a conservative approach.")
    elif expected_volatility < 0.35:
        explanation.append("This indicates a moderate-risk portfolio, suitable for a balanced approach.")
    else:
        explanation.append("This indicates a high-risk portfolio, suitable for investors with a more aggressive preference.")

    return "\n".join(explanation)

explanation = explain_min_var_portfolio(
    weights=optimal_weights,
    tickers=list(df_returns.columns),
    expected_return=expected_return,
    expected_volatility=expected_volatility,
)

print(explanation)


Major Holdings:
• JNJ makes up 80.34% of the portfolio — likely due to its lower volatility or better diversification.
• MSFT makes up 11.54% of the portfolio — likely due to its lower volatility or better diversification.
• AAPL makes up 5.94% of the portfolio — likely due to its higher volatility or lower diversification benefits.
• NVDA makes up 0.00% of the portfolio — likely due to its higher volatility or lower diversification benefits.
• TSLA makes up 2.18% of the portfolio — likely due to its higher volatility or lower diversification benefits.

Expected Portfolio Performance:
The expected annual return is 9.82%, with an annualized volatility of 18.79%.
This indicates a relatively low-risk portfolio, applicable for a conservative approach.


In [17]:
# CALCULATE PORTFOLIO PARAMETRIC VALUE AT RISK (VaR) USING COVARIANCE MATRIX
def portfolio_var(df_returns, weights, alpha=0.05):
    """
    Calculates 1-day parametric Value at Risk (VaR) for the entire portfolio.

    Parameters:
        df_returns (pd.DataFrame): Daily returns
        weights (list or np.ndarray): Portfolio weights (sum to 1)
        alpha (float): Significance level (e.g., 0.05 for 95% confidence)

    Returns:
        float: 1-day VaR as a positive percentage
    """

    weights = np.array(weights)
    z_score = norm.ppf(1 - alpha)  # Z-score for the given confidence level

    mu_p = np.dot(weights, df_returns.mean())  # Portfolio mean return
    sigma_p = np.sqrt(np.dot(weights, np.dot(df_returns.cov(), weights)))  # Portfolio volatility

    var_port = -(mu_p + z_score * sigma_p)  # VaR formula

    return var_port

# TESTING THE FUNCTION
var_portfolio_95 = portfolio_var(df_returns, optimal_weights, alpha=0.05)
print(f"Portfolio 1 day VaR at 95% confidence level: {var_portfolio_95:.2%}")

# INTERPRET RESULTS
print("\nInterpreting the results:")
print(f"There is a {alpha*100}% chance that the portfolio can lose more than {var_portfolio_95:.2%} of its value in one day.")


Portfolio 1 day VaR at 95% confidence level: -1.99%

Interpreting the results:
There is a 5.0% chance that the portfolio can lose more than -1.99% of its value in one day.


In [18]:
# CONVERT TO DOLLAR VAR
def dollar_var(portfolio_value, var_percentage):
    """
    Converts percentage VaR to dollar amount.

    Parameters:
        portfolio_value (float): Total value of the portfolio (e.g., $100,000)
        var_percentage (float): VaR as a decimal (e.g., 0.0243 for 2.43%)

    Returns:
        float: Dollar value at risk
    """
    return portfolio_value * var_percentage

# HYPOTHETICAL PORTFOLIO VALUE
portfolio_value = 100000  # Example: $100,000

# CALCULATE DOLLAR VAR USING MODEL OUTPUT
dollar_var_result = dollar_var(portfolio_value, var_portfolio_95)

# INTERPRETATION OUTPUT
print("\nInterpreting Portfolio-Level Value at Risk (95% Confidence):")
print(f"Based on your actual portfolio allocation, there is a 5% chance the portfolio could lose more than {var_portfolio_95:.2%} of its value in a single trading day.")
print(f"For a portfolio valued at ${portfolio_value:,.0f}, this translates to a potential 1-day loss exceeding ${dollar_var_result:,.2f}.")
print("This estimate assumes normal market conditions and historical return patterns.")



Interpreting Portfolio-Level Value at Risk (95% Confidence):
Based on your actual portfolio allocation, there is a 5% chance the portfolio could lose more than -1.99% of its value in a single trading day.
For a portfolio valued at $100,000, this translates to a potential 1-day loss exceeding $-1,985.89.
This estimate assumes normal market conditions and historical return patterns.


In [19]:

import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [20]:
# CREATE ETL PIPLINE TO PGADMIN RELATION DATABASE

# Load environment variables from .env file
load_dotenv()

# Extract credentials from environment variables
user = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
port = os.getenv("PG_PORT")
db = os.getenv("PG_DB")

# Check if credentials are loaded
if not all([user, password, host, port, db]):
    raise ValueError("Database credentials are not set in the environment variables.")


# Add time stamp to the DataFrame
timestamp = pd.Timestamp.now()


# Build DataFrame
summary_df = pd.DataFrame({
    'ticker': df_returns.columns,
    'sharpe_ratio': sharpe_ratios,
    'var_95': var_95,
    'var_param_95': var_param_95,
    'max_drawdown': max_dd,
    'rolling_vol_20d': rolling_vol.tail(1).T.values.flatten(),
    'optimal_weight': optimal_weights,
    'timestamp': timestamp,
})


# PostgreSQL connection parameters
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}')


# Export to SQL and overwrite existing table
try:

    summary_df.to_sql("risk_metrics", engine, if_exists='append', index=False)
    logger.info("Data exported to PostgreSQL database successfully, portfolio risk metrics with timestamps appended successfully.")

    expected_return, expected_volatility = portfolio_performance(df_returns, optimal_weights)
    var_portfolio_95 = portfolio_var(df_returns, optimal_weights, alpha=0.05)

    portfolio_summary_df = pd.DataFrame([{
        'timestamp': timestamp,
        'portfolio_return': expected_return,
        'portfolio_volatility': expected_volatility,
        'portfolio_var': var_portfolio_95
    }])

    portfolio_summary_df.to_sql("portfolio_summary", engine, if_exists='append', index=False)
    logger.info("Portfolio summary exported to PostgreSQL successfully.")

except Exception as e:
    logger.error(f"Error exporting data to PostgreSQL: {e}")
finally:
    engine.dispose()
    logger.info("PostgreSQL connection closed.")





2025-04-16 09:16:56,161 - INFO - Data exported to PostgreSQL database successfully, portfolio risk metrics with timestamps appended successfully.
2025-04-16 09:16:56,175 - INFO - Portfolio summary exported to PostgreSQL successfully.
2025-04-16 09:16:56,178 - INFO - PostgreSQL connection closed.
